## Final Project Submission

Please fill out: 
* Student name: **Alberto SYLVEUS** 
* Student pace: self paced / part time / full time
* Scheduled project review date/time: 
* Instructor name: **Wedter JEROME**
*  
[Click here to get the github repository](https://github.com/tolly00/dsc-phase2-myproject.git)


## Overview

In this project, we will use exploratory data analysis to generate insight for a business stackholder. 

## Business Understanding

After seing all the big company creating original video content and the want ti get in on the fun. A company has decided to create a new movie studio, but the personal doesn't know anything about creating movies. So we are going to exploring what types of films are currently doing the best at the box office. we will translate those findings into actionable insights that the head of the company's new movie studio can use to help decide what type of films to create.

## Data Understanding

* We have data from various locations, the different files have different formats. Some are compressed CSV(comma-separated values) or TSV(tab-separated values) files that can be opened using spreadsheet software or `pd.read_csv`, whith the data from IMDB is located in a SQLite database. In this case all those data are zippep in a folder that called `zippedData`, we will use `file explorer` to unzip them. Right now those files are ready to use.

    The goal of this analysis is to identify what types of films are currently doing the best at the box office to help the company's new movie studio about their decisions to create movie.

* In `zippedData` we are going to use `im.db` file. This is a sqlite database file. This database has 8 tables: movies_basics, directors, known_for, movies_akas, movies_rating, persons, principals, writers.
* The tables that have relevant data to make this analysis possible is: **movie_basics** and **movie_rating**.

  - **movie_basics:** has data about title, runtime, genres, etc...
  - **movie_rating**: has data about average rating and number of votes
* film with more votes

* film with more rating

* more common type film

* runtimes minutes with numvotes and average raating

## Data Preparation

In [1]:
# import necessary modules
import pandas as pd
import sqlite3

In [2]:
# connect to the database
path = "zippedData/im.db"
conn = sqlite3.connect(path)
q = """
    SELECT name
    FROM sqlite_master
    WHERE type = "table"
    """
pd.read_sql(q,conn) #show all tables name of the database

,name
0,movie_basics
1,directors
2,known_for
3,movie_akas
4,movie_ratings
5,persons
6,principals
7,writers


Let's use `movie_basics` and `movie_rating` tables to create a view.To avoid compromizing our analysis, we drop rows with NaN values.

In [7]:
# join them together
q = """
    CREATE VIEW IF NOT EXISTS movies AS
    SELECT
    b.movie_id, b.primary_title, b.start_year, b.genres, b.runtime_minutes, r.averagerating, r.numvotes 
    FROM movie_basics AS b
    JOIN movie_ratings AS r
    USING (movie_id)
    WHERE 
    b.genres IS NOT NULL AND b.genres != ''
    AND b.runtime_minutes IS NOT NULL
    AND r.averagerating IS NOT NULL
    AND r.numvotes IS NOT NULL;
    """
conn.execute(q)
conn.commit()

In [9]:
# show the movies view
q = """
SELECT * FROM movies;
"""
df_movies = pd.read_sql(q,conn)
df_movies.head()

,movie_id,primary_title,start_year,genres,runtime_minutes,averagerating,numvotes
0,tt0063540,Sunghursh,2013,"Action,Crime,Drama",175.0,7.0,77
1,tt0066787,One Day Before the Rainy Season,2019,"Biography,Drama",114.0,7.2,43
2,tt0069049,The Other Side of the Wind,2018,Drama,122.0,6.9,4517
3,tt0100275,The Wandering Soap Opera,2017,"Comedy,Drama,Fantasy",80.0,6.5,119
4,tt0137204,Joe Finds Grace,2017,"Adventure,Animation,Comedy",83.0,8.1,263


In [10]:
# A quickly preview on data type and NaN values
df_movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65720 entries, 0 to 65719
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   movie_id         65720 non-null  object 
 1   primary_title    65720 non-null  object 
 2   start_year       65720 non-null  int64  
 3   genres           65720 non-null  object 
 4   runtime_minutes  65720 non-null  float64
 5   averagerating    65720 non-null  float64
 6   numvotes         65720 non-null  int64  
dtypes: float64(2), int64(2), object(3)
memory usage: 3.5+ MB


In [4]:
# A quickly preview on data type and NaN values
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73856 entries, 0 to 73855
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   movie_id         73856 non-null  object 
 1   primary_title    73856 non-null  object 
 2   start_year       73856 non-null  int64  
 3   genres           73052 non-null  object 
 4   runtime_minutes  66236 non-null  float64
 5   averagerating    73856 non-null  float64
 6   numvotes         73856 non-null  int64  
dtypes: float64(2), int64(2), object(3)
memory usage: 3.9+ MB


## Analysis and results

### Business Recommendation 1

### Business Recommendation 2

### Business Recommendation 3

### Vizualisation

## Conclusion

## Next step